<a href="https://colab.research.google.com/github/alex8937/CS-10-601-Machine-Learning/blob/master/main_100_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! git clone https://github.com/alex8937/introtodeeplearning_labs.git
% cd introtodeeplearning_labs
! git pull
% cd ..

Cloning into 'introtodeeplearning_labs'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1124 (delta 0), reused 0 (delta 0), pack-reused 1120
Receiving objects: 100% (1124/1124), 135.29 MiB | 29.48 MiB/s, done.
Resolving deltas: 100% (624/624), done.
/content/introtodeeplearning_labs
Already up to date.
/content


In [0]:
import tensorflow as tf
import numpy as np
import os
tf.enable_eager_execution()
text = ''
for file in os.listdir('introtodeeplearning_labs/lab1/data/'):
  text += open(os.path.join('introtodeeplearning_labs/lab1/data', file), encoding="utf8").read()
print('Entire corpus length:', len(text))

In [4]:

vocab = sorted(set(text))
vocab_size = len(vocab)
print ('{} unique characters'.format(vocab_size))

char2idx = {c : idx for idx, c in enumerate(vocab)}
idx2char = np.array(vocab)

text_int = np.array([char2idx[c] for c in text])

char_dataset = tf.data.Dataset.from_tensor_slices(text_int)

seq_length = 100
n_samples = len(text) // seq_length
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)
dataset = sequences.map(lambda seq : (seq[:-1], seq[1:]))

batch_size = 64
n_mini_batches = n_samples // batch_size

#what if I don't do shuffle
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

rnn = tf.keras.layers.CuDNNLSTM


4184 unique characters


In [5]:

def build_model(vocab_size, embedding_size, rnn_units, batch_size):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size, embedding_size,
                                        batch_input_shape = [batch_size, None]))
    model.add(rnn(rnn_units, 
                  return_sequences=True, 
                  recurrent_initializer='glorot_uniform',
                  stateful=True))
    model.add(tf.keras.layers.Dense(vocab_size))
    return model

embedding_size = 256
rnn_units = 1024

model = build_model(vocab_size = vocab_size,
                    embedding_size=embedding_size,
                    rnn_units=rnn_units,
                    batch_size=batch_size)

#what if using softmax
def loss_fun(labels, logits):
    return tf.keras.backend.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer = tf.train.AdamOptimizer(),
              loss = loss_fun)


checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                       save_weights_only=True)



Instructions for updating:
Colocations handled automatically by placer.


In [7]:
from google.colab import drive
drive.mount('/content/gdrive')
drive_weight_path = "/content/gdrive/My Drive/Colab Notebooks/model_weights_100.h5"
if os.path.exists(drive_weight_path):
  model.load_weights(drive_weight_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [15]:
history = model.fit(dataset.repeat(), 
                    epochs=20, 
                    steps_per_epoch=n_mini_batches, 
                    callbacks=[checkpoint_callback])
model.save_weights(drive_weight_path)

Epoch 1/20
 18/191 [=>............................] - ETA: 1:04 - loss: 0.8409

KeyboardInterrupt: ignored

In [16]:
gen_model = build_model(vocab_size, embedding_size, rnn_units, batch_size=1)

gen_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

gen_model.build(tf.TensorShape([1, None]))



def generate_text(model, start_string, temperature = 1.0):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
    num_generate = 100

  # Converting our start string to numbers (vectorizing) 
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))


print(generate_text(gen_model, start_string=text[10165: 10167]), 1)


段誉一点头，说道：“这些日子倒不过我师父、师姊，我的话说明白了，我……我苦苦自己，不许你。”
段誉道：“出家之时，只怕难以收场。四五爷对头一眼之处，真要流星觉。”
众人听她语气中的菊剑忽然想起一事，左手袖 1


In [0]:
text[10165:11000]

'段誉哈哈大笑，拍手说道：“今日当真是大开眼界，叹为观止了。”\n龚光杰手忙脚乱的除下长裤，露出两条生满了黑毛的大腿。那少女叫道：“你这恶人爱欺侮人，叫你全身脱得精光，瞧你羞也不羞！”又是嘘嘘两声尖呼，那貂儿也真听话，爬上龚光杰左腿，立时钻入了他衬裤之中。练武厅上有不少女子，龚光杰这条衬裤是无论如何不肯脱的，双足乱跳，双手在自己小腹、屁股上拍了一阵，大叫一声，跌跌撞撞的往外直奔。\n他刚奔到厅门，忽然门外抢进一个人来，砰的一声，两人撞了个满怀。这一出一入，势道都是奇急，龚光杰踉跄后退，门外进来那人却仰天一交，摔倒在地。\n左子穆失声叫道：“容师弟！”\n龚光杰也顾不得裤中那只貂儿兀自从左腿爬到右腿、又从右腿爬上屁股，忙抢上将那人扶起，貂儿突然爬到了他前阴的要紧所在。他“啊”的一声大叫，双手忙去抓貂，那人又即摔倒。\n梁上少女格格娇笑，说道：“整得你也够了！”“嘶”的一下长声呼叫。貂儿从龚光杰裤中钻了出来，沿墙直上，奔到梁上，白影一闪，回到了那少女怀中。那少女赞道：“乖貂儿！”右手两根手指抓着一条小蛇的尾巴，倒提起来，在貂儿面前晃动。那貂儿前脚抓住，张口便吃，原来那少女手中这许多小蛇都是喂貂的食料。\n段誉前所未见，看得津津有味，见貂儿吃完一条小蛇，钻入了那少女腰间的皮囊。\n龚光杰再次扶起那人，惊叫：“容师叔，你……你怎么啦！”左子穆抢上前去，只见师弟容子矩双目圆睁，满脸愤恨之色，口鼻中却已没了气息。左子穆大惊，忙施推拿，已然无法救活。左子穆知道容子矩武功虽较已为逊，比龚光杰却高得多了，这么一撞，他居然没能避开，而一撞之下登时毙命，那定是进来之前已然身受重伤，忙解他上衣查察伤势。衣衫解开，只见他胸口赫然写着八个黑字：“神农帮诛灭无量剑”。众人不约而同的大声惊呼。\n这八个黑字深入肌理，既非墨笔书写，也不是用尖利之物刻划而致，竟是以剧毒的药物写就，腐蚀之下，深陷肌肤。\n左子穆略一凝视，不禁大怒，手中长剑一振，嗡嗡作响，喝道：“且瞧是神农帮诛灭无量剑，还是无量剑诛'